In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.linear_model import LogisticRegression
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import optuna
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
from warnings import filterwarnings
filterwarnings('ignore')

# Loading Data

In [3]:
BASE_DIR = Path("/kaggle/input/playground-series-s3e2/")

train = pd.read_csv(BASE_DIR / "train.csv").drop(columns="id")
test = pd.read_csv(BASE_DIR / "test.csv").drop(columns="id")

# Preprocessing

In [4]:
df = pd.concat([train.drop(columns=["stroke"]), test], axis=0).reset_index(drop=True)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked


# Encoding - One hot

In [5]:
df = pd.get_dummies(df)
df.head(3)

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,28.0,0,0,79.53,31.1,0,1,0,0,1,...,0,1,0,0,0,1,0,0,1,0
1,33.0,0,0,78.44,23.9,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
2,42.0,0,0,103.00,40.3,1,0,0,0,1,...,0,1,0,0,1,0,1,0,0,0


# Preprocessing v2

In [6]:
X = df.iloc[:-len(test), :]
test_new = df.iloc[-len(test):, :]
y = train.stroke

In [7]:
len(X) == len(train)

True

In [8]:
len(X), len(train)

(15304, 15304)

# Training all base models

In [9]:
# Tuned XGBoost Params
xgb_params = {'n_estimators': 272,
                 'max_depth': 4,
                 'learning_rate': 0.07360332417334109,
                 'min_child_weight': 1,
                 'gamma': 0.05180472121817407,
                 'subsample': 0.41981696676590474,
                 'colsample_bytree': 0.730818203141452,
                 'reg_alpha': 0.0005268315833160329,
                 'reg_lambda': 0.0007997627184403383}


# Tuned LightGBM Params
lgbm_params = {'n_estimators': 1942,
                 'num_rounds': 477,
                 'learning_rate': 0.2870761124159734,
                 'num_leaves': 37,
                 'max_depth': 6,
                 'min_data_in_leaf': 368,
                 'lambda_l1': 0.00017451291663562305,
                 'lambda_l2': 0.5090553595978456,
                 'min_gain_to_split': 0.02547126174774228,
                 'bagging_fraction': 0.20531850278394478,
                 'feature_fraction': 0.37437811030015083}


# Tuned Logistic Regression params
lr_params = {'solver': 'saga',
                 'C': 0.07655860626453208,
                 'tol': 0.00011268918749381429,
                 'max_iter': 484,
                 'penalty': 'l1',
                 'l1_ratio': 0.012007897837476486}


# Tuned CatBoost params
cat_params = {'loss_function': 'CrossEntropy',
                 'learning_rate': 0.7611165319863433,
                 'l2_leaf_reg': 0.014250721124770624,
                 'colsample_bylevel': 0.061218122768953775,
                 'depth': 1,
                 'boosting_type': 'Plain',
                 'bootstrap_type': 'Bernoulli',
                 'min_data_in_leaf': 18,
                 'one_hot_max_size': 14,
                 'subsample': 0.9972814280760031}

In [10]:
xgb_model = xgb.XGBClassifier(**xgb_params)
xgb_model.fit(X, y)

lgbm_model = lgbm.LGBMClassifier(objective="binary", is_unbalance=True, **lgbm_params)
lgbm_model.fit(X, y)

cat_model = catboost.CatBoostClassifier(**cat_params)
cat_model.fit(X, y)

lr_model = LogisticRegression(**lr_params)
lr_model.fit(X, y)

LogisticRegression(C=0.07655860626453208, l1_ratio=0.012007897837476486,
                   max_iter=484, penalty='l1', solver='saga',
                   tol=0.00011268918749381429)

### Let's add keras to the mix

In [11]:
inputs = layers.Input(shape=(21,))
x = layers.Dense(512, activation="relu")(inputs)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

x = layers.Dense(448, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

x = layers.Dense(32, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.7)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

keras_model = keras.Model(inputs=inputs, outputs=outputs)

optim = keras.optimizers.Adam(learning_rate= 0.0035)

keras_model.compile(optimizer=optim,
                   loss=keras.losses.binary_crossentropy,
                   metrics=[keras.metrics.AUC()])

2023-01-15 20:38:59.695099: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [12]:
keras_model.fit(X, y, epochs=48, verbose=0)

2023-01-15 20:39:18.263052: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [ ]:
# random forest just finished tuning, so lets add it as well

rf_params = {'n_estimators': 390,
                 'max_features': 14,
                 'min_samples_split': 19,
                 'min_samples_leaf': 2,
                 'max_depth': 5}

rf_model = RandomForestClassifier(**rf_params)

rf_model.fit(X, y)

In [ ]:
# def objective(trial):
#     w1 = round(trial.suggest_discrete_uniform("w_keras", 0.0, 1.0, 0.05), 2)
#     w2 = round(trial.suggest_discrete_uniform("w_xgb", 0.0, 1.0 - w1, 0.05),2)
#     w3 = round(trial.suggest_discrete_uniform("w_lgbm", 0.0, 1.0 - w1 - w2, 0.05),2)
#     w4 = round(trial.suggest_discrete_uniform("w_cat", 0.0, 1.0 - w1 - w2 - w3, 0.05), 2)
#     w5 = round(trial.suggest_discrete_uniform("w_rf", 0.0, 1.0 - w1 - w2 - w3 - w4, 0.05), 2)
#     w6 = 1.0 - w1 - w2 - w3 - w4 - w5
    
#     print(f"sum is {sum([w1, w2, w3, w4, w5, w6])}")
#     assert sum([w1, w2, w3, w4, w5, w6]) == 1.0
    
#     cv = KFold(n_splits=5, shuffle=True, random_state=1337)

#     cv_scores = np.empty(5)
#     for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
#         X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#         y_train, y_test = y[train_idx], y[test_idx]
        
#         keras_model = keras.fit(X_train, y_train, epochs=48)
#         keras
        
#         xgb_model = xgb.XGBRegressor(**xgb_params)
#         xgb_model.fit(X_train, y_train)
#         y_preds_xgb = xgb_model.predict_proba(X_test)
        
#         lgbm_model = lgbm.LGBMRegressor(objective="regression", **lgbm_params)
#         lgbm_model.fit(X_train, y_train)
#         y_preds_lgbm = lgbm_model.predict(X_test)
        
#         catboost_model = CatBoostRegressor(**catboost_params)
#         catboost_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=100)
#         y_preds_cat = catboost_model.predict(X_test)
        
#         y_preds_final = y_preds_xgb * w1 + y_preds_lgbm * w2 + y_preds_cat * w3
        
#         rmse = mse(y_test, y_preds_final, squared=False)
        
#         cv_scores[idx] = rmse
    
#     avg_rmse = np.mean(cv_scores)
#     return avg_rmse

In [ ]:
# weights based on vibes, beacuse too sleepy to actually get some meaningful it's 1:46 in the morning and i gotta sleep bad

In [13]:
keras_preds = keras_model.predict(test_new)[:, 0]
xgb_preds = xgb_model.predict_proba(test_new)[:, 1]
lgbm_preds = lgbm_model.predict_proba(test_new)[:, 1]
cat_preds = cat_model.predict_proba(test_new)[:, 1]
rf_preds = cat_model.predict_proba(test_new)[:, 1]
lr_preds = cat_model.predict_proba(test_new)[:, 1]

In [14]:
y_final = (keras_preds * 0.4) + (cat_preds * 0.15) + (xgb_preds * 0.15) + (lgbm_preds * 0.15) + (rf_preds * 0.1) + (lr_preds * 0.05)

In [16]:
# since we dropped id column from our test dataframe earlier but we need it for submission file so we're reloading it
test_idx = pd.read_csv(BASE_DIR / "test.csv").id
test_idx

0        15304
1        15305
2        15306
3        15307
4        15308
         ...  
10199    25503
10200    25504
10201    25505
10202    25506
10203    25507
Name: id, Length: 10204, dtype: int64

In [17]:
submission = pd.DataFrame(data={"id": test_idx, "stroke": y_final})
submission.head()

,id,stroke
0,15304,0.056831
1,15305,0.218428
2,15306,0.000380
3,15307,0.146781
4,15308,0.005680


In [18]:
submission.to_csv("submission.csv", index=False)